# Lab 3 - N-gram Language Model

#### 1. Implement a generalized N-gram language model for the Romanian language using a smoothing technique at your choice, build on a corpus you collect from the web and lemmatize.

In [ ]:
import nltk

In [131]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.util import everygrams
from nltk.lm import Laplace, NgramCounter
from nltk import ngrams
from nltk.probability import FreqDist

In [132]:
from datasets import load_dataset
ronec = load_dataset("ronec")
corpus = ronec["train"]
tokens_list = corpus["tokens"]

In [133]:
print(tokens_list[0])

['-', 'Iohannis', ',', 'Klaus', 'Vacanță', 'Iohannis', '-', 'căruia', 'puțin', 'îi', 'pasă', 'că', '40%', 'din', 'populația', 'țării', 'trăiește', 'sub', 'limita', 'sărăciei', 'altfel', 'nu', 'ar', 'putea', 'să', 'susțină', 'sifonarea', 'profiturilor', 'de', 'către', 'multinaționalele', 'străine', 'și', 'ar', 'susține', 'impozitul', 'pe', 'venit', '-', 'Băsescu', ',', 'Traian', 'Băsescu', 'care', 'a', 'spus', 'că', 'aplicarea', 'unui', 'impozit', 'pe', 'venit', 'ține', 'de', '„', 'Paleoliticul', 'Financiar', '”', 'semn', 'că', 'joacă', 'în', 'continuare', 'cum', 'îi', 'cântă', 'Sistemul', 'și', 'că', 'i', 'se', 'fâlfâie', 'de', 'Sifonarea', 'Profiturilor', 'și', 'de', 'milioanele', 'de', 'români', 'săraci']


In [138]:
flattened_tokens = [token for tokens in tokens_list for token in tokens]
lemmatized_tokens = [WordNetLemmatizer().lemmatize(token) for token in flattened_tokens]

In [139]:
n = 2 

n_grams = list(ngrams(lemmatized_tokens, n))
lm = Laplace(order=n)
lm.fit([n_grams], vocabulary_text=lemmatized_tokens)

#### 2. Compute the probability of a new Romanian sentence, given at input, using the n-gram model you developed.

In [140]:
sentence = "Astazi va ninge la munte."
tokens = [WordNetLemmatizer().lemmatize(token) for token in word_tokenize(sentence)]

n = 2  
new_ngrams = list(ngrams(tokens,n))

sentence_probability = 1.0  
for ngram in new_ngrams:
    probability = lm.score(ngram[-1], ngram[:-1])
    sentence_probability *= probability

print("Probability of the sentence:", sentence_probability)

Probability of the sentence: 1.0221789423720594e-22


#### 3.Use a pre-trained neural language model to predict the next two words after a sequence of 4 words given as input.

In [ ]:
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

In [145]:
input_text = "I am going to"

In [146]:
indexed_tokens = tokenizer.encode(input_text)
tokens_tensor = torch.tensor([indexed_tokens])

In [147]:
with torch.no_grad():
     for _ in range(0,2):
        outputs = model(tokens_tensor)
        predictions = outputs[0]
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        indexed_tokens.append(predicted_index)
        tokens_tensor = torch.tensor([indexed_tokens])
        
predicted_text = tokenizer.decode(indexed_tokens)

print(predicted_text)

 I am going to be a
